In [21]:
import os
import base64
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append("/home/karan.verma")
from verification_guardrails import get_type_of_text

In [22]:
userid = 'Transcriber@transcription-prod'
passwd = 'TranscribePass1'
ipaddr = 'transcription-prod.mysql.database.azure.com'
db_name = 'speech_transcribe'
dest_sql = "mysql+mysqldb://" + userid + ":" + passwd + "@" + ipaddr + "/" + db_name + "?charset=utf8&use_unicode=0"
sql_engine = create_engine(dest_sql,pool_recycle=3600)
db_connection    = sql_engine.connect()


In [23]:
data_from_transcribe = pd.read_sql('select * from data_transcribe where tag in (285,287,284) and verified in (1,8,11,7,17,18) and verified_time>="2020-08-01"', db_connection)


In [24]:
data_prev = pd.read_sql('select * from vs_data_transcribe_till_27082020', db_connection)


In [25]:
uniq_data = data_from_transcribe[~data_from_transcribe.text_id.isin(data_prev.text_id)]
uniq_data.describe()

,text_id,uploaded,verified,user_id,tag,verified_by,length,confidence,pre_processed,file_id,seq_id,test_data,lang,lang_lm,current_month_verification
count,1.218903e+06,1218903.0,1.218903e+06,1.218903e+06,1.218903e+06,1.218903e+06,1.218903e+06,1.218903e+06,1218903.0,1.218903e+06,1218903.0,1.218903e+06,1218903.0,1.218903e+06,1218903.0
mean,4.670723e+07,1.0,6.221705e+00,6.794799e+03,2.845782e+02,4.791734e+03,5.982303e+03,6.818184e-01,0.0,4.998495e+07,0.0,2.937559e-01,1.0,4.102049e-06,1.0
std,1.275884e+06,0.0,5.346629e+00,2.585813e+01,6.481868e-01,1.069185e+03,2.185817e+03,7.512742e-02,0.0,2.886999e+07,0.0,1.688570e+00,0.0,3.734563e-03,0.0
min,4.346808e+07,1.0,1.000000e+00,6.719000e+03,2.840000e+02,0.000000e+00,5.760000e+02,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,1.0,0.000000e+00,1.0
25%,4.624059e+07,1.0,1.000000e+00,6.775000e+03,2.840000e+02,5.019000e+03,4.572000e+03,6.334305e-01,0.0,2.499571e+07,0.0,0.000000e+00,1.0,0.000000e+00,1.0
50%,4.680796e+07,1.0,7.000000e+00,6.789000e+03,2.850000e+02,5.043000e+03,5.112000e+03,6.908850e-01,0.0,4.996480e+07,0.0,0.000000e+00,1.0,0.000000e+00,1.0
75%,4.772766e+07,1.0,8.000000e+00,6.818000e+03,2.850000e+02,5.070000e+03,6.516000e+03,7.384320e-01,0.0,7.501491e+07,0.0,0.000000e+00,1.0,0.000000e+00,1.0
max,4.871040e+07,1.0,1.800000e+01,6.859000e+03,2.870000e+02,5.094000e+03,1.544400e+04,8.741070e-01,0.0,9.999974e+07,0.0,1.000000e+01,1.0,4.000000e+00,1.0


In [26]:
uniq_data['corrected_utf_text_decoded'] = uniq_data['corrected_utf_text'].apply(base64.urlsafe_b64decode).str.decode('utf8', errors='ignore').dropna()
uniq_data['type_of_text'] = uniq_data['corrected_utf_text_decoded'].apply(get_type_of_text)



<ipython-input-26-274e80410cf1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniq_data['corrected_utf_text_decoded'] = uniq_data['corrected_utf_text'].apply(base64.urlsafe_b64decode).str.decode('utf8', errors='ignore').dropna()
<ipython-input-26-274e80410cf1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniq_data['type_of_text'] = uniq_data['corrected_utf_text_decoded'].apply(get_type_of_text)


In [9]:
uniq_data_usable =  uniq_data[(uniq_data['type_of_text']==2) | (uniq_data['type_of_text']==0) ]

uniq_data_usable_hr = uniq_data_usable["length"].sum()/(60*1000*60)
print("Total Audio Duration in hours  :- ", uniq_data_usable_hr)


Total Audio Duration in hours  :-  1513.8482522222223


In [10]:
uniq_data_usable["length"].count()

953644

# Making Final DB

In [27]:
data_from_transcribe['corrected_utf_text_decoded'] = data_from_transcribe['corrected_utf_text'].apply(base64.urlsafe_b64decode).str.decode('utf8', errors='ignore').dropna()
data_from_transcribe['type_of_text'] = data_from_transcribe['corrected_utf_text_decoded'].apply(get_type_of_text)


In [28]:
data_for_sec_non_prom = data_from_transcribe[data_from_transcribe.type_of_text==2]
data_for_sec_non_prom.describe()

,text_id,uploaded,verified,user_id,tag,verified_by,length,confidence,pre_processed,file_id,seq_id,test_data,lang,lang_lm,current_month_verification,type_of_text
count,5.959410e+05,595941.0,595941.000000,595941.000000,595941.000000,595941.000000,595941.000000,595941.000000,595941.0,5.959410e+05,595941.0,595941.00000,595941.0,595941.000000,595941.0,595941.0
mean,4.621096e+07,1.0,10.229771,6794.560519,284.572991,4752.155829,5962.562749,0.675095,0.0,5.001899e+07,0.0,0.21581,1.0,0.000002,1.0,2.0
std,1.345069e+06,0.0,4.971315,25.036478,0.611685,1132.586143,2029.426084,0.073665,0.0,2.887343e+07,0.0,1.45311,0.0,0.001295,0.0,0.0
min,4.346836e+07,1.0,7.000000,6719.000000,284.000000,0.000000,972.000000,0.000000,0.0,0.000000e+00,0.0,0.00000,1.0,0.000000,1.0,2.0
25%,4.539815e+07,1.0,7.000000,6775.000000,284.000000,5019.000000,4572.000000,0.626925,0.0,2.498176e+07,0.0,0.00000,1.0,0.000000,1.0,2.0
50%,4.632953e+07,1.0,7.000000,6789.000000,285.000000,5042.000000,5184.000000,0.683664,0.0,5.003469e+07,0.0,0.00000,1.0,0.000000,1.0,2.0
75%,4.712266e+07,1.0,18.000000,6817.000000,285.000000,5069.000000,6552.000000,0.732660,0.0,7.506560e+07,0.0,0.00000,1.0,0.000000,1.0,2.0
max,4.871040e+07,1.0,18.000000,6859.000000,287.000000,5094.000000,15444.000000,0.874107,0.0,9.999923e+07,0.0,10.00000,1.0,1.000000,1.0,2.0


In [29]:
data_for_clean = uniq_data[uniq_data.type_of_text==0]
data_for_clean.describe()

,text_id,uploaded,verified,user_id,tag,verified_by,length,confidence,pre_processed,file_id,seq_id,test_data,lang,lang_lm,current_month_verification,type_of_text
count,5.504630e+05,550463.0,550463.000000,550463.000000,550463.000000,550463.000000,550463.000000,550463.000000,550463.0,5.504630e+05,550463.0,550463.000000,550463.0,550463.000000,550463.0,550463.0
mean,4.669316e+07,1.0,2.976413,6794.669028,284.543682,4791.810536,5482.998425,0.699237,0.0,4.997120e+07,0.0,0.271372,1.0,0.000002,1.0,0.0
std,1.294556e+06,0.0,3.292571,25.994223,0.641027,1069.082757,1433.830908,0.061217,0.0,2.885000e+07,0.0,1.624832,0.0,0.001348,0.0,0.0
min,4.346809e+07,1.0,1.000000,6719.000000,284.000000,21.000000,936.000000,0.000000,0.0,0.000000e+00,0.0,0.000000,1.0,0.000000,1.0,0.0
25%,4.622294e+07,1.0,1.000000,6774.000000,284.000000,5016.000000,4572.000000,0.659393,0.0,2.499917e+07,0.0,0.000000,1.0,0.000000,1.0,0.0
50%,4.680442e+07,1.0,1.000000,6789.000000,284.000000,5042.000000,5076.000000,0.708084,0.0,4.993331e+07,0.0,0.000000,1.0,0.000000,1.0,0.0
75%,4.773987e+07,1.0,8.000000,6818.000000,285.000000,5069.000000,6012.000000,0.747205,0.0,7.496474e+07,0.0,0.000000,1.0,0.000000,1.0,0.0
max,4.871039e+07,1.0,11.000000,6859.000000,287.000000,5094.000000,15444.000000,0.874107,0.0,9.999974e+07,0.0,10.000000,1.0,1.000000,1.0,0.0


In [31]:
data_for_sec_non_prom.drop(data_for_sec_non_prom.columns[[-1]], axis = 1, inplace = True) 
data_for_sec_non_prom.columns

Index(['text_id', 'uploaded', 'verified', 'user_id', 'tag', 'verified_by',
       'length', 'confidence', 'uploaded_time', 'verified_time',
       'last_modified_time', 'base_dir', 'utf_text', 'corrected_utf_text',
       'hindi_text', 'audio_file', 'hindi_txt', 'pre_processed', 'file_id',
       'seq_id', 'test_data', 'lang', 'lang_lm', 'auto_lang',
       'current_month_verification', 'srv_session_id'],
      dtype='object')

In [33]:
data_for_clean.drop(data_for_clean.columns[[-1]], axis = 1, inplace = True) 
data_for_clean.columns

Index(['text_id', 'uploaded', 'verified', 'user_id', 'tag', 'verified_by',
       'length', 'confidence', 'uploaded_time', 'verified_time',
       'last_modified_time', 'base_dir', 'utf_text', 'corrected_utf_text',
       'hindi_text', 'audio_file', 'hindi_txt', 'pre_processed', 'file_id',
       'seq_id', 'test_data', 'lang', 'lang_lm', 'auto_lang',
       'current_month_verification', 'srv_session_id'],
      dtype='object')

In [36]:
data_for_sec_non_prom['length']/= 10
data_for_sec_non_prom['length']= data_for_sec_non_prom['length'].astype('int')
pd.set_option('display.max_columns',80)
data_for_sec_non_prom.head()

<ipython-input-36-abc0053f4da6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_sec_non_prom['length']/= 10
<ipython-input-36-abc0053f4da6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_sec_non_prom['length']= data_for_sec_non_prom['length'].astype('int')


,text_id,uploaded,verified,user_id,tag,verified_by,length,confidence,uploaded_time,verified_time,last_modified_time,base_dir,utf_text,corrected_utf_text,hindi_text,audio_file,hindi_txt,pre_processed,file_id,seq_id,test_data,lang,lang_lm,auto_lang,current_month_verification,srv_session_id
1,43541443,1,7,6830,284,5081,511,0.752223,NaT,2020-09-08 08:00:37,2020-09-08 02:30:37,b'livtranscription/transcription-gluster-audio/',b'4KSX4KWH4KSuIOCkluClh-CksuCkqOClhyDgpLXgpL7g...,b'QCBfIEAg4KSX4KWH4KSuIOCkluClh-CksuCkqOClhyDg...,None,b'api_EN_S_11Jun20/app_589055128_3765565884391...,None,0,91751680,0,0,1,0,b'NA',1,None
10,43541492,1,7,6785,284,5040,968,0.741656,NaT,2020-09-07 12:51:34,2020-09-07 07:21:34,b'livtranscription/transcription-gluster-audio/',b'4KSs4KSa4KWN4KSa4KWL4KSCIOCkleClhyDgpLLgpL_g...,b'QCBfIEAg4KSs4KSa4KWN4KSa4KWL4KSCIOCkleClhyDg...,None,b'api_EN_S_11Jun20/app_1576782380_376556510586...,None,0,61820416,0,0,1,0,b'NA',1,None
11,43541494,1,7,6815,284,5052,457,0.773628,NaT,2020-09-28 13:43:18,2020-09-28 08:13:18,b'livtranscription/transcription-gluster-audio/',b'4KSo4KWH4KSyIOCkquClieCksuCkv-Cktg==',b'4KSo4KWH4KSyIOCkquClieCksuCkv-CktiBAIF8gQA==',None,b'api_EN_S_11Jun20/app_1409175437_376556507790...,None,0,2589952,0,0,1,0,b'NA',1,None
20,43541516,1,18,6785,284,5040,392,0.722872,NaT,2020-09-07 12:51:33,2020-09-07 07:21:33,b'livtranscription/transcription-gluster-audio/',b'4KSX4KSw4KWN4KSy4KWN4KS4IOCktuClguCknA==',b'QCBfIEAg4KSX4KSw4KWN4KSy4KWN4KS4IOCktuClguCk...,None,b'api_EN_S_11Jun20/app_311785740_3765564708266...,None,0,66967040,0,0,1,0,b'NA',1,None
27,43541536,1,7,6815,284,5029,921,0.575483,NaT,2020-09-23 15:55:20,2020-09-23 10:25:20,b'livtranscription/transcription-gluster-audio/',b'4KS44KWB4KSq4KSwIOCkq-CkvuCkh-CktSDgpJ_gpY3g...,b'4KST4KSq4KWN4KSq4KWLIOCkjyDgpKvgpL7gpIfgpLUg...,None,b'api_EN_S_11Jun20/app_210496138_3765564417945...,None,0,45632768,0,0,1,0,b'NA',1,None


In [37]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd

tableName   = "data_transcribe_sec_non_prom_12102020"

userid = 'Transcriber@transcription-prod'
passwd = 'TranscribePass1'
ipaddr = 'transcription-prod.mysql.database.azure.com'
db_name = 'speech_transcribe'
dest_sql = "mysql+mysqldb://" + userid + ":" + passwd + "@" + ipaddr + "/" + db_name + "?charset=utf8&use_unicode=0"
sql_engine = create_engine(dest_sql,pool_recycle=3600)
db_connection    = sql_engine.connect()

try:
    frame = data_for_sec_non_prom.to_sql(tableName, db_connection, if_exists='fail');
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table %s created successfully."%tableName);   
finally:
    db_connection.close()


Table data_transcribe_sec_non_prom_12102020 created successfully.


In [38]:
data_for_clean['length']/= 10
data_for_clean['length']= data_for_clean['length'].astype('int')
pd.set_option('display.max_columns',80)
data_for_clean.head()

<ipython-input-38-49e164c04dbf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_clean['length']/= 10
<ipython-input-38-49e164c04dbf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_clean['length']= data_for_clean['length'].astype('int')


,text_id,uploaded,verified,user_id,tag,verified_by,length,confidence,uploaded_time,verified_time,last_modified_time,base_dir,utf_text,corrected_utf_text,hindi_text,audio_file,hindi_txt,pre_processed,file_id,seq_id,test_data,lang,lang_lm,auto_lang,current_month_verification,srv_session_id
0,43541441,1,1,6815,284,5052,457,0.746853,NaT,2020-09-28 13:43:18,2020-09-28 08:13:18,b'livtranscription/transcription-gluster-audio/',b'4KSu4KS-4KSw4KWN4KSV4KWN4KS4',b'4KSu4KS-4KS44KWN4KSV',None,b'api_EN_S_11Jun20/app_1830714103_376556594442...,None,0,20630528,0,0,1,0,b'NA',1,None
2,43541444,1,1,6815,284,5052,842,0.667935,NaT,2020-09-28 13:43:18,2020-09-28 08:13:18,b'livtranscription/transcription-gluster-audio/',b'4KSW4KS_4KSy4KWM4KSo4KWHIOCkteCkvuCksuClgCDg...,b'4KSW4KS_4KSy4KWM4KSo4KWHIOCkteCkvuCksuClgCDg...,None,b'api_EN_S_11Jun20/app_9344400_376556587567598...,None,0,75987968,0,0,1,0,b'NA',1,None
5,43541466,1,1,6830,284,5081,554,0.820730,NaT,2020-09-08 08:00:37,2020-09-08 02:30:37,b'livtranscription/transcription-gluster-audio/',b'4KSX4KWC4KSX4KSyIOCkquCkv-CkleCljeCkuOClh-Ck...,b'4KSX4KWC4KSX4KSyIOCkquCkv-CkleCljeCkuOClh-Ck...,None,b'api_EN_S_11Jun20/app_1409328655_376556557638...,None,0,87231744,0,0,1,0,b'NA',1,None
6,43541477,1,1,6830,284,5081,457,0.641958,NaT,2020-09-08 08:00:37,2020-09-08 02:30:37,b'livtranscription/transcription-gluster-audio/',b'4KSX4KS-4KSh4KS84KWA',b'4KSX4KS-4KSh4KS84KWA',None,b'api_EN_S_11Jun20/app_9344400_376556544468608...,None,0,86086144,0,0,1,0,b'NA',1,None
8,43541489,1,8,6785,284,5040,457,0.841090,NaT,2020-09-07 12:51:34,2020-09-07 07:21:34,b'livtranscription/transcription-gluster-audio/',b'4KS14KSoIOCkquCljeCksuCkuCDgpK7gpYvgpKzgpL7g...,b'4KS14KSoIOCkquCljeCksuCkuCDgpK7gpYvgpKzgpL7g...,None,b'api_EN_S_11Jun20/app_1409328655_376556515031...,None,0,13054464,0,0,1,0,b'NA',1,None


In [39]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd

tableName   = "vs_data_transcribe_clean_12102020"

userid = 'Transcriber@transcription-prod'
passwd = 'TranscribePass1'
ipaddr = 'transcription-prod.mysql.database.azure.com'
db_name = 'speech_transcribe'
dest_sql = "mysql+mysqldb://" + userid + ":" + passwd + "@" + ipaddr + "/" + db_name + "?charset=utf8&use_unicode=0"
sql_engine = create_engine(dest_sql,pool_recycle=3600)
db_connection    = sql_engine.connect()

try:
    frame = data_for_clean.to_sql(tableName, db_connection, if_exists='fail');
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table %s created successfully."%tableName);   
finally:
    db_connection.close()


Table vs_data_transcribe_clean_12102020 created successfully.
